# Applied Project in Big Data on Industrial Dataset

## DATA COLLECTION AND PROCESSING TECHNIQUES
## Part V. Spark advanced data processing

### 1. Libraries and Spark setup

In [ ]:
import os
import sys
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [ ]:
print('user:', os.environ['JUPYTERHUB_SERVICE_PREFIX'])

def uiWebUrl(self):
    from urllib.parse import urlparse
    web_url = self._jsc.sc().uiWebUrl().get()
    port = urlparse(web_url).port
    return '{}proxy/{}/jobs/'.format(os.environ['JUPYTERHUB_SERVICE_PREFIX'], port)

SparkContext.uiWebUrl = property(uiWebUrl)

conf = SparkConf()
conf.set('spark.master', 'local[*]')
conf.set('spark.driver.memory', '16G')
conf.set('spark.driver.maxResultSize', '8G')
sc = SparkContext(conf=conf)
spark = SparkSession(sc)
spark

In [ ]:
WORK_PATH = '/home/jovyan/__RAYPFP24'


def access_data(file_path):
    with open(file_path) as file:
        access_data = json.load(file)
    return access_data


access_s3_data = access_data(f'{WORK_PATH}/.access_jhub_data')

In [ ]:
spark._jsc.hadoopConfiguration().set('fs.s3a.access.key', access_s3_data['aws_access_key_id'])
spark._jsc.hadoopConfiguration().set('fs.s3a.secret.key', access_s3_data['aws_secret_access_key'])
spark._jsc.hadoopConfiguration().set('fs.s3a.impl','org.apache.hadoop.fs.s3a.S3AFileSystem')
spark._jsc.hadoopConfiguration().set('fs.s3a.multipart.size', '104857600')
spark._jsc.hadoopConfiguration().set('fs.s3a.block.size', '33554432')
spark._jsc.hadoopConfiguration().set('fs.s3a.threads.max', '256')
spark._jsc.hadoopConfiguration().set('fs.s3a.endpoint', 'http://storage.yandexcloud.net')
spark._jsc.hadoopConfiguration().set('fs.s3a.aws.credentials.provider', 
                                     'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')

### 2. Data load with Spark

#### 2.1. Direct path to files

In [ ]:
files_path = f'{WORK_PATH}/data/events'
files_mask = f'{files_path}/data_2023-08-29.csv'  # test part of data
sdf = spark.read.option('escape','"').csv(files_mask, header=True)

In [ ]:
sdf.printSchema()

In [ ]:
sdf.count()

#### 2.2. Through bucket connect

In [ ]:
files_path = f'data/events'
files_mask = f'{files_path}/data_2023-08-29.csv'  # test part of data
sdf = spark.read.option('escape','"').csv(
    f's3a://{access_s3_data["bucket_name"]}/{files_mask}', 
    header=True
)

In [ ]:
sdf.printSchema()

In [ ]:
sdf.count()

#### 2.3. More data

In [ ]:
files_path = 'data/events'
files_mask = f'{files_path}/data_2023-08-0*.csv'  # first decade for the month
sdf = spark.read.option('escape','"').csv(
    f's3a://{access_s3_data["bucket_name"]}/{files_mask}', 
    header=True
)

In [ ]:
%%time
sdf.count()

In [ ]:
sdf = sdf.withColumn('event_datetime', F.to_timestamp('event_datetime'))

In [ ]:
sdf.limit(5).toPandas()

### 3. Basic EDA

#### 3.1. Users and devices

In [ ]:
sdf.select('profile_id').distinct().count()

In [ ]:
sdf.select('appmetrica_device_id').distinct().count()

#### 3.2. Events

In [ ]:
sdf.select('event_json').distinct().count()

In [ ]:
events = sdf.select('event_json').distinct().collect()
events[:10]

In [ ]:
event_names = sdf.select('event_name').distinct().collect()
len(event_names)

### 4. Data processing

#### 4.1. Single user example

In [ ]:
# find at least one user with payments
df = sdf.filter(
    sdf.event_name.like('%Оплата/Завешили оплату%') 
    | sdf.event_name.like('%Оплата/Завершили оплату%') 
).limit(5).toPandas()
df.head()

In [ ]:
sample_id = df.profile_id[0]
print(sample_id)

In [ ]:
sdf_sample = sdf.filter(sdf.profile_id == sample_id)

In [ ]:
sdf_sample.count()

In [ ]:
sdf_sample.limit(5).toPandas()

In [ ]:
event_names = sdf_sample.select('event_name').distinct().collect()
len(event_names)

In [ ]:
event_names

##### 4.1.1. History for single user

In [ ]:
user_grouped = sdf_sample.groupBy('event_name').count()

In [ ]:
user_grouped.toPandas()

In [ ]:
sdf_sample.select(
    'event_datetime',
    'event_name'
).orderBy(F.col('event_datetime'), ascending=False).toPandas()

##### 4.1.2. Lags dataset for single user

In [ ]:
w_10sec = (Window()
      .partitionBy(F.col('profile_id'))
      .orderBy(F.col('event_datetime').cast('timestamp').cast('long'))
      .rangeBetween(-11, -1))
w_10_to_60sec = (Window()
      .partitionBy(F.col('profile_id'))
      .orderBy(F.col('event_datetime').cast('timestamp').cast('long'))
      .rangeBetween(-61, -12))
w_60sec_to_5min = (Window()
      .partitionBy(F.col('profile_id'))
      .orderBy(F.col('event_datetime').cast('timestamp').cast('long'))
      .rangeBetween(-301, -62))

In [ ]:
sdf_ds = (
    sdf_sample
        .withColumn('lag_10sec', F.collect_list('event_name').over(w_10sec))
        .withColumn('lag_10_to_60sec', F.collect_list('event_name').over(w_10_to_60sec))
        .withColumn('lag_60sec_to_5min', F.collect_list('event_name').over(w_60sec_to_5min))
        .select(
            'event_datetime',
            'event_name',
            'lag_10sec',
            'lag_10_to_60sec',
            'lag_60sec_to_5min'
        )
    .orderBy(F.col('event_datetime'), ascending=False)
)
sdf_ds.limit(5).toPandas()

In [ ]:
sdf_ds.printSchema()

In [ ]:
#ev1 = 'Проверка/История платежей'
ev1 = 'Мои штрафы/Документы/Есть СТС'
sdf_ds = (
    sdf_ds
        .withColumn(
            'lag_60sec_ev1', 
            F.when(F.array_contains("lag_10sec", ev1), 1).otherwise(0)
        )
        .withColumn(
            'lag_10_to_60sec_ev1', 
            F.when(F.array_contains("lag_10_to_60sec", ev1), 1).otherwise(0)
        )
        .withColumn(
            'lag_60sec_to_5min_ev1', 
            F.when(F.array_contains("lag_60sec_to_5min", ev1), 1).otherwise(0)
        )
)
sdf_ds.limit(5).toPandas()

In [ ]:
sdf_ds.filter(sdf_ds.lag_60sec_to_5min_ev1 == 1).limit(5).toPandas()

In [ ]:
def count_ev(lst, ev=''):
    return lst.count(ev)

In [ ]:
from pyspark.sql.types import IntegerType

# Converting function to UDF 
count_ev1_udf = F.udf(lambda x: count_ev(x, ev=ev1), IntegerType())

In [ ]:
sdf_ds = (
    sdf_ds
        .withColumn(
            'lag_60sec_to_5min_ev1', 
            count_ev1_udf(F.col("lag_60sec_to_5min"))
        )
)
sdf_ds.limit(10).toPandas()

In [ ]:
sdf_ds.filter(sdf_ds.lag_60sec_to_5min_ev1 > 1).limit(5).toPandas()